In [1]:
import pandas as pd
import numpy as np

import datetime as dt

import matplotlib.pylab as plt

from scipy import interpolate
from scipy.stats import norm
from scipy.optimize import brentq
from scipy.integrate import quad

In [2]:
# Discount Rate
rate_df = pd.read_csv('zero_rates_20201201.csv')

# SPX General Data
SPX_df = pd.read_csv('SPX_options.csv')
SPX_df['strike_price'] = SPX_df['strike_price']/1000
SPX_df['mid_price'] = (SPX_df['best_bid'] + SPX_df['best_offer'])/2

# SPX Maturity Data
SPX = SPX_df[(SPX_df.exdate == 20210115)]

# Time To Maturity
today = dt.date(2020, 12, 1)
exdate = dt.date(2021, 1, 15)
T = (exdate-today).days/365.0

# Discount Rate Interpolation
x = rate_df['days']
y = rate_df['rate']
f = interpolate.interp1d(x,y)
r = f(T*365)/100

S = 3662.45
K = 3660


# Implied Volatility - SPX

In [3]:
# Black-Scholes Model
def BlackScholesCall(S, K, r, Sigma, T):
    d1 = (np.log(S/K)+(r+Sigma**2/2)*T) / (Sigma*np.sqrt(T))
    d2 = d1 - Sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BlackScholesPut(S, K, r, Sigma, T):
    return BlackScholesCall(S,K,r,Sigma,T)- S + K*np.exp(-r*T)

In [4]:
# Implied European Options Volatility Model
def impliedCallVolatility(S, K, r, price, T):
    try:
        impliedVol = brentq(lambda x: price -
                        BlackScholesCall(S, K, r, x, T),
                        1e-6, 1)
    except Exception:
        impliedVol = np.nan
 
    return impliedVol

def impliedPutVolatility(S, K, r, price, T):
    try:
        impliedVol = brentq(lambda x: price -
                        BlackScholesPut(S, K, r, x, T),
                        1e-6, 1)
    except Exception:
        impliedVol = np.nan

    return impliedVol

## At The Money Volatility (ExDate: 2021/1/15)
ATM_call = SPX[(SPX.strike_price == K)]
ATM_call = ATM_call[(ATM_call.cp_flag == "C")]
ATM_put = SPX[(SPX.strike_price == K)]
ATM_put = ATM_put[(ATM_put.cp_flag == "P")]
Sigma_call = impliedCallVolatility(S, K, r , ATM_call.mid_price, T)
Sigma_put = impliedPutVolatility(S, K, r, ATM_put.mid_price, T)
Sigma = (Sigma_call + Sigma_put)/2

In [5]:
Sigma

0.18537188428747395

# Black Scholes

In [6]:
# Black Scholes Model Integrated Variance Function
def BSM_callintegrand(K, S, r, T, Sigma):
    price = BlackScholesCall(S, K, r, Sigma, T) / K**2
    return price

def BSM_putintegrand(K, S, r, T, Sigma):
    price = BlackScholesPut(S, K, r, Sigma, T) / K**2
    return price



In [7]:
F = S * np.exp(r*T)
BSM_I_put = quad(lambda x: BSM_putintegrand(x, S, r, T, Sigma), 0.0, F)
BSM_I_call = quad(lambda x: BSM_callintegrand(x, S, r, T, Sigma), F, 5000)
BSM_Var = 2*np.exp(r*T)*(BSM_I_put[0] + BSM_I_call[0])

print('Black Scholes Model Integrated Variance: %.9f' % BSM_Var)

Black Scholes Model Integrated Variance: 0.004236501


In [8]:
# Black Scholes Model Derivative Pricing Model
BSM_Sigma = np.sqrt(BSM_Var/T)

def BSM_pricing(S,r,T,Sigma):
    price = S ** (1/3) * np.exp((1/3) * r * T - (1/9) *Sigma ** 2 * T) + 1.5 * (np.log(S) + r * T - 0.5 * (Sigma ** 2) * T) + 10
    return price

BSM_price = BSM_pricing(S,r,T,BSM_Sigma)

print('Black-Scholes Model Derivating Pricing: %.4f' % BSM_price)

Black-Scholes Model Derivating Pricing: 37.7144


# Bachelier Model

In [9]:
# Bachelier Model
def BachelierCall(S, K, r, Sigma, T):
    d = (S-K) / (S*Sigma*np.sqrt(T))
    disc = np.exp(-r*T)
    return disc*((S-K)*norm.cdf(d)+S*Sigma*np.sqrt(T)*norm.pdf(d))

def BachelierPut(S, K, r, Sigma, T):
    d = (S-K) / (S*Sigma*np.sqrt(T))
    disc = np.exp(-r*T)
    return disc*((K-S)*norm.cdf(-d)+S*Sigma*np.sqrt(T)*norm.pdf(-d))

# Bachelier Model Integrated Variance Function
def BACHcallintegrand(K, S, r, T, Sigma):
    price = BachelierCall(S, K, r, Sigma, T) / K**2
    return price

def BACHputintegrandb(K, S, r, T, Sigma):
    price = BachelierPut(S, K, r, Sigma, T) / K**2
    return price

In [10]:
F = S * np.exp(r*T)
BACH_I_put = quad(lambda x: BACHputintegrandb(x, S, r, T, Sigma), 0.0, F)
BACH_I_call = quad(lambda x: BACHcallintegrand(x, S, r, T, Sigma), F, 5000)
BACH_Var = 2*np.exp(r*T)*(BACH_I_put[0] + BACH_I_call[0])

print('Bachelier Model Integrated Variance: %.9f' % BACH_Var)

Bachelier Model Integrated Variance: 0.004263876


In [11]:
# Bachelier Model Derivative Pricing Model
BACH_Sigma = np.sqrt(BACH_Var/T)

BACH = lambda x: (((S + BACH_Sigma*np.sqrt(T)*x)**(1/3))+ (1.5* np.log(S + BACH_Sigma*np.sqrt(T)*x)) +10 )*np.exp(-(x**2/2))
x_BACH,err = quad(BACH, -(np.inf), np.inf)

BACH_Price = np.exp(-r*T)/np.sqrt(2*np.pi)*x_BACH

print('Bachelier Model Derivative Pricing: %.4f' % BACH_Price)

Bachelier Model Derivative Pricing: 37.7136


# SABR Model

In [12]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrSigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrSigma = numer/denom

    return sabrSigma

def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, K, r, sabr_vol, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, K, r, sabr_vol, T)


def SABRcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price


def SABRputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price

In [13]:
alpha = 1.817
beta = 0.7
rho = -0.404
nu = 2.790

F = S * np.exp(r*T)
SABR_I_put = quad(lambda x: SABRputintegrand(x, S, r, T, alpha, beta, rho, nu), 0, F)
SABR_I_call = quad(lambda x: SABRcallintegrand(x, S, r, T, alpha, beta, rho, nu), F, 5000)
SABR_Var = 2*np.exp(r*T)*(SABR_I_put[0] + SABR_I_call[0])

print('SABR Model Integrated Variance: %.9f' % SABR_Var)

SABR Model Integrated Variance: 0.006337325


In [14]:
# SABR Model Derivative Pricing Model
SABR_Sigma = np.sqrt(SABR_Var/T)

x_BACH_SABR = lambda x: (((S + SABR_Sigma*np.sqrt(T)*x)**(1/3))+ (1.5* np.log(S + SABR_Sigma*np.sqrt(T)*x)) +10 )*np.exp(-(x**2/2))
x_SABR,Error = quad(x_BACH_SABR, -(np.inf), np.inf)

BACH_SABR_Price = np.exp(-r*T)/np.sqrt(2*np.pi)*x_SABR

print('Bachelier Model Derivative Pricing (SABR): %.4f' % BACH_SABR_Price)

BSM_SABR_Price = BSM_pricing(S,r,T,SABR_Sigma)

print('Black-Scholes Model Derivative Pricing (SABR): %.4f' % BSM_SABR_Price)

Bachelier Model Derivative Pricing (SABR): 37.7136
Black-Scholes Model Derivative Pricing (SABR): 37.7092
